In [108]:
import deap
import cma
from deap import cma

from deap import creator , base , tools, algorithms
import numpy as np
import random

# Introduction a DEAP : 
DEAP est une librairie qui permet de prototyper rapidement des algorithmes évolutionnaires et des algorithmes génétiques, elle contient des modes de perturbations, de sélection et de croisement plus ou moins sophistiqués qui sont mis a disposition sous forme d'outils.
Dans cette partie nous allons essayer d'aller en compliquant les choses tout en présentant les différents outils implémentés.

## Création de types : 
Dans deap nous pouvons créer dynamiquement des types en utilisant le créateur. \
Généralement nous n'avons pas beaucoup de types a utiliser sauf pour la programmation génétique, on utilisera également "base" pour manipuler plusieurs classes de bases proposées par DEAP.

In [158]:
"""
La fonction create permet de créer un type, elle prend en paramètres un nom, une classe de base et des attributs
chaque
"""

"""
Les premiers type que l'on va créer serrons des fonctions de fitness, pour se faire nous utiliserons la classe 
Fitness présente dans base et comme attributs additionnels nous ajouterons l'attribut weight qui permet de : 
    - Préciser les poids des différents attributs pour les départager si besoin est
    - Préciser le sens dans lequel on optimise i.e si on souhaite les maximiser ou les minimiser
"""
creator.create("Fitness_Function", base.Fitness, weights=(-1.0,))
creator.create("Multi_Objectif", base.Fitness, weights=(-1.0, 1.0))
# Pour utiliser un type crée on ecris par exemple creator.Fitness_Function

"""
A présent on va créer des individus, pour ce faire nous utiliserons généralement comme classe de base 
la classe liste que nous enrichirons par un attribut fitness qui aura un des types crées précédements.
Dans la partie sur la programmation génétique on verra d'autres types plus exotiques.
"""
creator.create("Individual", list, fitness=creator.Fitness_Function)
#On aurait pu utiliser la classe ndarray dispo sur numpy comme classe de base
creator.create("Individual2", np.ndarray , fitness=creator.Fitness_Function)


/home/mohamed/.local/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Fitness_Function' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/mohamed/.local/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Multi_Objectif' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/mohamed/.local/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual2' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


## Création et remplissage de la Toolbox : 
La toolbox est l'élément le plus important d'un algorithme génétique, elle permet de spécifier les différentes fonctions que l'on va utiliser pour perturber les solutions, pour les faire muter etc...
Son remplissage nécessite d'utiliser la fonction register, et la encore pour la remplire nous pourrons nous appuyer sur les outils mis a disposition par deap.\
La toolbox sert également a préciser la structure de la population et comment l'initialiser.

In [62]:
"""
D'abord on crée la population et on l'ajoute a la ToolBox 
"""
#Taille de l'individu 
IND_SIZE = 10
#Initialisation de la toolbox
toolbox = base.Toolbox()
#On enregistre la fonction qui sert a initialiser les individus 
toolbox.register("initialisation_random", random.random)
#On ajoute les individus en précisant le mode d'initialisation, le type la fonction d'initialisation et leur taille 
toolbox.register("population", tools.initRepeat, creator.Individual,toolbox.initialisation_random, n=IND_SIZE)

In [63]:
"""
Maintenant je vais présenter chaque outils
"""

"""
1/Fonction d'évaluation : 
"""
#On crée un individu pour tester en utilisant "individual"

individu1 = toolbox.population()
print(individu1)
print(individu1.fitness) #Fitness pas encore calculée
print(individu1.fitness.valid) #Quand la fitness n'est pas calculée on dit qu'elle est invalide

#D'abord comment enregistrer une fonction d'évaluation


def evaluate(individual):
    #Votre fonction d'évaluation qui retourne une ou plusieurs valeurs (autant qu'il n'y en a dans weights)
    return 2,

individu1.fitness.values = evaluate(individu1)
#Aprés évaluation 
print (individu1.fitness.valid) #ça passe a valid parce qu'on a rempli le champ values
print (individu1.fitness.values)

#Maintenant on enregistre dans la toolbox comme ça

[0.030775733631281943, 0.1055280897663714, 0.4560934343237334, 0.07831634383058061, 0.7735217725034221, 0.10435798367588867, 0.7117756770989118, 0.43782485845080255, 0.3353257190829486, 0.026895086572969462]
()
False
True
(2.0,)


In [64]:
"""
2/Opérateur de mutation (perturbation) : 
"""
mutant = toolbox.clone(individu1)
individu2, = tools.mutGaussian(mutant, mu=0.0, sigma=0.2, indpb=0.7)

del mutant.fitness.values
print(individu1)
print(individu2)
print(individu1.fitness.valid)
print(individu2.fitness.valid)

[0.030775733631281943, 0.1055280897663714, 0.4560934343237334, 0.07831634383058061, 0.7735217725034221, 0.10435798367588867, 0.7117756770989118, 0.43782485845080255, 0.3353257190829486, 0.026895086572969462]
[0.34549912199603805, -0.048531188742857306, 0.4560934343237334, 0.14746749441031015, 0.8329300203601051, 0.17102111197943634, 0.7117756770989118, 0.4326993728095312, 0.3353257190829486, 0.026895086572969462]
True
False


In [65]:
"""
3/Opérateur de croisement (intensification) : 
"""
child1, child2 = [toolbox.clone(ind) for ind in (individu1, individu2)]
tools.cxBlend(child1, child2, 0.8)
del child1.fitness.values
del child2.fitness.values
print(child1)
print(child2)

[0.15895240630498414, -0.08659543316156883, 0.4560934343237334, 0.10663841998069948, 0.7769070310679638, 0.16317535977954994, 0.7117756770989118, 0.4339712056552733, 0.3353257190829486, 0.026895086572969462]
[0.21732244932233583, 0.14359233418508294, 0.4560934343237334, 0.11914541826019127, 0.8295447617955635, 0.11220373587577508, 0.7117756770989118, 0.43655302560506043, 0.3353257190829486, 0.026895086572969462]


In [104]:
"""
4/Opérateur de selection (intensification) : 
"""
Lambd = 3
MU = 10
selected = tools.selBest([child1, child2], 1)
print(selected)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


Maintenant on assemble le tout dans une seule bulle ce qui nous fait notre premier algorithme évolutionnaire, nous considérons le design suivant : 
- Une stratégie de perturbation qui consiste a inverser un bit.
- Une sélection par tournoi
- Un croisement Binaire qui consiste en l'inversion de deux points.

Le probléme que nous allons résoudre pour voire si ça marche est le problème de maximum binaire, la fonction d'évaluation serra donc naturellement la somme des éléments de la liste et on s'attend a la fin a trouver un vecteur de 1.

In [174]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator 
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, 100)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# the goal ('fitness') function to be maximized
def evaluation_func(individual):
    return sum(individual),


toolbox.register("evaluate", evaluation_func)


# ----------
# Ajout des opérateurs 
# ----------
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


# ----------

random.seed(64)
# create an initial population of 300 individuals (where
# each individual is a list of integers)
pop = toolbox.population(n=300)
"""
Supposons qu'on adopte un mécanisme connu consistant a assigner des probabilités : 
    -CXPB : Probabilités de croisements.
    -MUTPB : Probabilités de mutations.
Ne pas oublier que les mutation diversifient la recherche donc souvent il ne faut pas en abuser pour une bonne
convergence
"""
CXPB, MUTPB = 0.5, 0.2

print("Start of evolution")
# Evaluer la population
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit
    

print("  Evaluated %i individuals" % len(pop))
# Met les fitnesses dans une liste
fits = [ind.fitness.values[0] for ind in pop]
gen = 0
# Begin the evolution
while max(fits) < 100 and g < 1000:
    # A new generation
    gen = gen + 1
    print("---- Iteration : %i ----" % gen)
    
    # On sélectionne des individus (supposons que tous les éléments parents participent a la génération de l'offspring)
    offspring = toolbox.select(pop, len(pop))
    
    # On les clone etant donné que les oppérateurs aggissent in place et on récupére une liste de références
    offspring = list(map(toolbox.clone, offspring))
    
    # on croise les éléments paires avec les éléments impaires suivant une probabilité CXPB
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            #On déléte les fitness, vu que les éléments ont changés elle doit être recalculée
            del child1.fitness.values
            del child2.fitness.values
            
    
    for mutant in offspring:
        #Mutation avec probabilité MUTPB
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    print("  Evaluated %i individuals" % len(invalid_ind))
    # The population is entirely replaced by the offspring
    pop[:] = offspring
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]
    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x * x for x in fits)
    std = abs(sum2 / length - mean ** 2) ** 0.5
    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)
print("-- End of (successful) evolution --")
best_ind = tools.selBest(pop, 1)[0]
print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

Start of evolution
  Evaluated 300 individuals
---- Iteration : 1 ----
  Evaluated 181 individuals
  Min 44.0
  Max 66.0
  Avg 54.833333333333336
  Std 4.349584909952722
---- Iteration : 2 ----
  Evaluated 191 individuals
  Min 47.0
  Max 68.0
  Avg 58.45666666666666
  Std 3.455641120769904
---- Iteration : 3 ----
  Evaluated 199 individuals
  Min 52.0
  Max 68.0
  Avg 60.95333333333333
  Std 2.9024970092816367
---- Iteration : 4 ----
  Evaluated 167 individuals
  Min 47.0
  Max 71.0
  Avg 62.96
  Std 2.907186497858939
---- Iteration : 5 ----
  Evaluated 175 individuals
  Min 57.0
  Max 73.0
  Avg 64.99
  Std 2.8489588741621903
---- Iteration : 6 ----
  Evaluated 168 individuals
  Min 58.0
  Max 74.0
  Avg 66.93333333333334
  Std 2.8051539866624524
---- Iteration : 7 ----
  Evaluated 187 individuals
  Min 59.0
  Max 76.0
  Avg 68.91666666666667
  Std 2.826609669236565
---- Iteration : 8 ----
  Evaluated 171 individuals
  Min 62.0
  Max 76.0
  Avg 70.88666666666667
  Std 2.4455038108513

In [175]:
"""
Un peu trop long ? 
Il existe un autre moyen de coder ces comportements basiques (tellement basiques que même moi j'ai copié des
partie de ce code du code de mon prof qui l'a copié du code de la librairie) et c'est en utilisant le package
algorithms.
"""
pop = toolbox.population(n=300)
log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=1000)

gen	nevals
0  	300   
1  	179   
2  	156   
3  	168   
4  	150   
5  	178   
6  	178   
7  	165   
8  	170   
9  	180   
10 	140   
11 	185   
12 	168   
13 	158   
14 	173   
15 	158   
16 	164   
17 	159   
18 	176   
19 	166   
20 	171   
21 	177   
22 	162   
23 	175   
24 	172   
25 	176   
26 	156   
27 	176   
28 	171   
29 	175   
30 	192   
31 	159   
32 	171   
33 	164   
34 	147   
35 	173   
36 	179   
37 	164   
38 	159   
39 	166   
40 	162   
41 	178   
42 	164   
43 	162   
44 	168   
45 	171   
46 	163   
47 	155   
48 	171   
49 	171   
50 	168   
51 	148   
52 	161   
53 	171   
54 	188   
55 	162   
56 	178   
57 	159   
58 	170   
59 	140   
60 	157   
61 	150   
62 	154   
63 	159   
64 	176   
65 	146   
66 	176   
67 	167   
68 	178   
69 	162   
70 	168   
71 	165   
72 	177   
73 	155   
74 	165   
75 	172   
76 	156   
77 	185   
78 	190   
79 	175   
80 	178   
81 	171   
82 	169   
83 	174   
84 	163   
85 	164   
86 	177   
87 	180   
88 	153   
89 	145   

751	160   
752	162   
753	182   
754	142   
755	166   
756	160   
757	168   
758	179   
759	157   
760	163   
761	158   
762	173   
763	179   
764	177   
765	184   
766	174   
767	158   
768	171   
769	168   
770	159   
771	166   
772	172   
773	167   
774	163   
775	153   
776	143   
777	155   
778	168   
779	161   
780	175   
781	165   
782	153   
783	173   
784	162   
785	149   
786	153   
787	165   
788	147   
789	169   
790	160   
791	169   
792	166   
793	168   
794	174   
795	158   
796	141   
797	170   
798	175   
799	165   
800	160   
801	157   
802	162   
803	177   
804	184   
805	153   
806	180   
807	155   
808	162   
809	152   
810	171   
811	156   
812	172   
813	172   
814	160   
815	181   
816	171   
817	164   
818	140   
819	152   
820	175   
821	179   
822	170   
823	178   
824	175   
825	145   
826	159   
827	154   
828	155   
829	155   
830	170   
831	159   
832	168   
833	172   
834	187   
835	163   
836	148   
837	151   
838	165   
839	165   
840	159   
841	159   

In [176]:
"""
Un peu triste comme display...
On peut le customiser en demandant a l'algorithme de calculer des statistiques grace au package stats.
"""
pop = toolbox.population(n=300)
#Initialiser l'objet stats en lui précisant une fonction lui permettant de savoir pour chaque individu sur quoi
#il est sensé calculer ces stats.
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
#On ajoute des statistiques que l'on veut avoir 
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
#Pour l'executer sur une population on pourrait faire compile
#Mais la le but est de le donner a l'algorithme
log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=100,stats=stats)

gen	nevals	avg  	std    	min	max
0  	300   	49.58	4.75993	38 	66 
1  	193   	53.6233	4.10789	40 	66 
2  	189   	56.7367	3.68067	48 	69 
3  	205   	59.67  	3.77197	53 	70 
4  	207   	62.3067	3.63492	49 	72 
5  	186   	64.8567	3.32708	54 	73 
6  	192   	67.11  	3.0262 	58 	75 
7  	185   	69.2567	2.69025	59 	75 
8  	160   	70.7467	2.67379	61 	78 
9  	200   	72.3567	2.81711	60 	80 
10 	177   	73.8   	2.63059	64 	80 
11 	203   	74.84  	2.82862	67 	82 
12 	188   	76.49  	2.91603	67 	85 
13 	196   	77.9467	2.82084	69 	85 
14 	188   	79.21  	2.85527	70 	85 
15 	181   	80.6567	2.62529	72 	86 
16 	199   	81.74  	2.78072	72 	88 
17 	196   	83.0267	2.52968	73 	90 
18 	189   	83.9367	2.8471 	73 	92 
19 	188   	85.0433	2.62198	76 	92 
20 	200   	85.87  	2.91544	76 	93 
21 	215   	87.0367	3.08901	77 	95 
22 	196   	88.4333	2.78308	79 	95 
23 	194   	89.4033	2.90069	78 	96 
24 	194   	90.48  	2.68383	81 	97 
25 	216   	91.4433	2.82137	82 	97 
26 	202   	92.6733	2.73373	84 	99 
27 	186   	93.55  	2.785

In [177]:
"""
On retrouve également les autres algorithmes évolutionnaires connus.
"""
pop = toolbox.population(n=300)
#Décomenter pour tester si vous voulez tester
#l= algorithms.eaMuPlusLambda(pop, toolbox, mu=300, lambda_=100, cxpb=0.5, mutpb=0.3, ngen=200, stats=stats)
#l= algorithms.eaMuCommaLambda(pop, toolbox, mu=300, lambda_=400, cxpb=0.5, mutpb=0.3, ngen=200, stats=stats)

On peut également manipuler des stratégies, par exemple CMA-ES qu'on a vu précédemment a été développé en utilisant deap et par conséquent on peut utiliser cette stratégie :
- generate permet de générer une population a partir d'un élément.
- update permet d'actualiser a partir d'une population évaluée les paramètres permettant de generer des éléments

In [180]:

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
def evaluation_func(individual):
    return sum(individual),
toolbox.register("evaluate", evaluation_func)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

pop = toolbox.population(n=300)

S = cma.Strategy(centroid = pop[0],sigma = 0.01)
toolbox.register("generate", S.generate, creator.Individual)
toolbox.register("update",S.update)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
#On ajoute des statistiques que l'on veut avoir 
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("max", np.max)

logbook = tools.Logbook()



for g in range(100):
    pop = toolbox.generate()
    evaluate(pop)
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    toolbox.update(pop)
    record = stats.compile(pop)
    logbook.record(gen=g, **record)
    logbook.header = "gen", "avg", "max"
    print(logbook.stream)

"""
Quelqu'un peut expliquer pourquoi ça marche pas ??
"""


gen	avg    	max    
0  	51.9846	52.0964
1  	52.0789	52.3162
2  	52.1843	52.3693
3  	52.3029	52.5104
4  	52.5067	52.6553
5  	52.6175	52.8626
6  	52.7568	52.904 
7  	52.8695	52.9666
8  	52.9532	53.0846
9  	53.0583	53.2112
10 	53.2136	53.4023
11 	53.2989	53.4234
12 	53.378 	53.5486
13 	53.4898	53.6618
14 	53.5993	53.8327
15 	53.7401	53.902 
16 	53.8572	54.0632
17 	54.0082	54.2383
18 	54.14  	54.3157
19 	54.2665	54.5201
20 	54.4612	54.764 
21 	54.6271	54.9733
22 	54.8011	55.0614
23 	55.0212	55.2607
24 	55.2349	55.4805
25 	55.4069	55.623 
26 	55.6212	55.9035
27 	55.8768	56.1091
28 	56.0822	56.3054
29 	56.2197	56.539 
30 	56.4304	56.7694
31 	56.6346	56.8776
32 	56.7201	57.1014
33 	56.8774	57.182 
34 	56.947 	57.3346
35 	57.2122	57.6693
36 	57.5913	58.0597
37 	57.9432	58.3549
38 	58.2935	58.6731
39 	58.665 	59.2761
40 	59.0729	59.4147
41 	59.4202	59.9408
42 	59.7228	60.18  
43 	60.0974	61.0473
44 	60.5837	61.1414
45 	61.1216	61.6927
46 	61.4814	62.1473
47 	61.9917	62.565 
48 	62.3199	62.9853


### A Nous de jouer : 
On va essayer cette fois ci de prototyper rapidement un algorithme génétique de notre choix sur le dataset boston en utilisant les mêmes fonctions d'évaluations que précédément et le même réseau de neurones.
Vous choisissez l'algorithme que vous voulez et on voit qui a le meilleur résultat en 1000 itérations.

In [159]:
#La fonction d'évaluation : 
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
def evaluation_function(w):
    X, y = load_boston(return_X_y=True)
    Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
    Network.set_parameters(w)
    y_pred = Network.predict(X)
    return mean_squared_error(y,y_pred),

#Je vous remet la classe NeuralNetwork pour rappel (et pour éviter de l'importer)
def sigmoid(x):
    return 1./(1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

class SimpleNeuralControllerNumpy():
    def __init__(self, n_in, n_out, n_hidden_layers=2, n_neurons_per_hidden=5, params=None):
        self.dim_in = n_in
        self.dim_out = n_out
        # if params is provided, we look for the number of hidden layers and neuron per layer into that parameter (a dicttionary)
        if (not params==None):
            if ("n_hidden_layers" in params.keys()):
                n_hidden_layers=params["n_hidden_layers"]
            if ("n_neurons_per_hidden" in params.keys()):
                n_neurons_per_hidden=params["n_neurons_per_hidden"]
        self.n_per_hidden = n_neurons_per_hidden
        self.n_hidden_layers = n_hidden_layers
        self.weights = None 
        self.n_weights = None
        self.init_random_params()
        self.out = np.zeros(n_out)
        #print("Creating a simple mlp with %d inputs, %d outputs, %d hidden layers and %d neurons per layer"%(n_in, n_out,n_hidden_layers, n_neurons_per_hidden))
    def init_random_params(self):
        if(self.n_hidden_layers > 0):
            self.weights = [np.random.random((self.dim_in,self.n_per_hidden))] # In -> first hidden
            self.bias = [np.random.random(self.n_per_hidden)] # In -> first hidden
            for i in range(self.n_hidden_layers-1): # Hidden -> hidden
                self.weights.append(np.random.random((self.n_per_hidden,self.n_per_hidden)))
                self.bias.append(np.random.random(self.n_per_hidden))
            self.weights.append(np.random.random((self.n_per_hidden,self.dim_out))) # -> last hidden -> out
            self.bias.append(np.random.random(self.dim_out))
        else:
            self.weights = [np.random.random((self.dim_in,self.dim_out))] # Single-layer perceptron
            self.bias = [np.random.random(self.dim_out)]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])

    def get_parameters(self):
        """
        Returns all network parameters as a single array
        """
        flat_weights = np.hstack([arr.flatten() for arr in (self.weights+self.bias)])
        return flat_weights

    def set_parameters(self, flat_parameters):
        """
        Set all network parameters from a single array
        """
        i = 0 # index
        to_set = []
        self.weights = list()
        self.bias = list()
        if(self.n_hidden_layers > 0):
            # In -> first hidden
            w0 = np.array(flat_parameters[i:(i+self.dim_in*self.n_per_hidden)])
            self.weights.append(w0.reshape(self.dim_in,self.n_per_hidden))
            i += self.dim_in*self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                w = np.array(flat_parameters[i:(i+self.n_per_hidden*self.n_per_hidden)])
                self.weights.append(w.reshape((self.n_per_hidden,self.n_per_hidden)))
                i += self.n_per_hidden*self.n_per_hidden
            # -> last hidden -> out
            wN = np.array(flat_parameters[i:(i+self.n_per_hidden*self.dim_out)])
            self.weights.append(wN.reshape((self.n_per_hidden,self.dim_out)))
            i += self.n_per_hidden*self.dim_out
            # Samefor bias now
            # In -> first hidden
            b0 = np.array(flat_parameters[i:(i+self.n_per_hidden)])
            self.bias.append(b0)
            i += self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                b = np.array(flat_parameters[i:(i+self.n_per_hidden)])
                self.bias.append(b)
                i += self.n_per_hidden
            # -> last hidden -> out
            bN = np.array(flat_parameters[i:(i+self.dim_out)])
            self.bias.append(bN)
            i += self.dim_out
        else:
            n_w = self.dim_in*self.dim_out
            w = np.array(flat_parameters[:n_w])
            self.weights = [w.reshape((self.dim_in,self.dim_out))]
            self.bias = [np.array(flat_parameters[n_w:])]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])
    
    def predict(self,x):
        if(self.n_hidden_layers > 0):
            #Input
            a = np.matmul(x,self.weights[0]) + self.bias[0]
            #y = sigmoid(a)
            y = a
            # hidden -> hidden
            for i in range(1,self.n_hidden_layers-1):
                a = np.matmul(y, self.weights[i]) + self.bias[i]
                y = sigmoid(a)
            # Out
            a = np.matmul(y, self.weights[-1]) + self.bias[-1]
            #out = tanh(a)
            out = a
            return out
        else: # Simple monolayer perceptron
            #return tanh(np.matmul(x,self.weights[0]) + self.bias[0])
            return np.matmul(x,self.weights[0]) + self.bias[0]
        
#Et voila pour rappel comment on initialise les paramètres aléatoirement
Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
Network.init_random_params()
print(Network.get_parameters())

[0.99185358 0.7292189  0.81732141 0.85757774 0.30308554 0.75508946
 0.17731125 0.74376942 0.62139551 0.04663265 0.20134711 0.93657372
 0.2349595  0.37518375]


In [1]:
#Allez y, essayez
#proposez une EA Simple ici  


In [ ]:
#Ici proposez plûtot quelque chose inspiré de CMA (on sait que ça fonctionne bien)
